In [1]:
import pandas as pd
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import psycopg2
import scipy

pd.set_option('display.max_columns', None)

try:
    conn = psycopg2.connect("dbname='capstone' user='samkreter' host='localhost'")
except:
    print("I am unable to connect to the database")

cur = conn.cursor()

sql = "select   fights.f1result, \
                f1.height_inches AS f1_height,\
                f1.reach_inches AS f1_reach, \
                f1.weight_lbs AS f1_weight, \
                f1.strike_offense_per_min AS f1_strike_offense_per_min, \
                f1.strike_offense_accuracy AS f1_strike_offense_accuracy, \
                f1.strike_defense_per_min AS f1_strike_defense_per_min, \
                f1.strike_defense_accuracy AS f1_strike_defense_accuracy, \
                f1.takedowns_per_fight AS f1_takedowns_per_fight, \
                                                \
                f2.height_inches AS f2_height, \
                f2.reach_inches AS f2_reach, \
                f2.weight_lbs AS f2_weight, \
                f2.strike_offense_per_min AS f2_strike_offense_per_min, \
                f2.strike_offense_accuracy AS f2_strike_offense_accuracy, \
                f2.strike_defense_per_min AS f2_strike_defense_per_min, \
                f2.strike_defense_accuracy AS f2_strike_defense_accuracy, \
                f2.takedowns_per_fight AS f2_takedowns_per_fight \
                from octagon.fights INNER JOIN octagon.fighters AS f1 ON octagon.fights.f1name = f1.name INNER JOIN octagon.fighters AS f2 ON octagon.fights.f2name = f2.name;"

df = pd.read_sql_query(sql, conn)

test = pd.read_sql_query("select * from octagon.fighters",conn)



# df = pd.read_sql_query("select * from octagon.fights",conn)
df2 = pd.read_sql_query("select * from octagon.fighters",conn)
conn.close()


In [3]:
test.head(n=1)

,fid,name,dob,height_inches,weight_lbs,association,division,country,reach_inches,strike_offense_per_min,strike_offense_accuracy,strike_defense_per_min,strike_defense_accuracy,takedowns_per_fight,takedowns_accuracy,takedowns_defense,submissions_per_fight,wins,losses,draws,total_fights,win_pct,finish_pct
0,230,John Lewis,1969-07-16,72,160,Lewis,Welterweight,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,0


In [6]:
len(test['association'].unique())

634

In [ ]:
df['f1result'] = df['f1result'].replace(['win', 'draw', 'NC'],[1,2,2])
df.head()

,f1result,f1_height,f1_reach,f1_weight,f1_strike_offense_per_min,f1_strike_offense_accuracy,f1_strike_defense_per_min,f1_strike_defense_accuracy,f1_takedowns_per_fight,f2_height,f2_reach,f2_weight,f2_strike_offense_per_min,f2_strike_offense_accuracy,f2_strike_defense_per_min,f2_strike_defense_accuracy,f2_takedowns_per_fight
0,1,78,81,260,2.82,34,2.46,46,3.26,79,81,265,0.85,44,2.48,52,2.12
1,1,70,NaN,151,NaN,NaN,NaN,NaN,NaN,70,73,170,1.79,36,2.21,63,2.51
2,1,72,NaN,170,1.89,36,3.23,52,0.62,68,66,155,2.39,38,2.37,57,2.64
3,1,69,74,247,0.97,30,2.87,59,1.06,74,NaN,235,2.61,69,3.91,26,4.34
4,1,72,73,170,1.61,41,2.00,46,4.17,74,NaN,170,2.55,40,2.34,35,0.00


### Interpolate the data to fill in the NAs to get a better result of the data
It was found that dropna works the best for the highest correclation results, We will revisit this later

In [ ]:
df0 = df.interpolate()
df0 = df.dropna()
df0 = df.interpolate(method='spline', order=2)
df0 = df.interpolate(method='pchip')
df = df.interpolate(method='polynomial', order=2)


## Move the lossers to the end to have data for losing people

In [ ]:
df2 = df.copy()
tmp = df2.ix[:,1:9]


df2.ix[:,1:9] = df2.ix[:,9:17].values
df2.ix[:,9:17] = tmp.values
df2['f1result'] = 0


df2.head()

### Combine the two dataframes and randomize

In [ ]:
df = df.append(df2)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
#Pull the labels out of the dataset
y = df['f1result']
del df['f1result']

X = df

In [ ]:
from sklearn import datasets
# Our Classifier
from sklearn.naive_bayes import GaussianNB
# Get the accuracy score of the final model
from sklearn.metrics import accuracy_score
# Split the training set into train and test
from sklearn.cross_validation import train_test_split
#MLP Classifer
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state = 1)

names = ["MLP","Naive Bayes","KNN","SVM Linear","SVM gamma","Decsion Tree", "Random Forest","adaBoost"]

clfs = [MLPClassifier(solver='sgd', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1),
        GaussianNB(),
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025),
        SVC(gamma=2, C=1),
        #GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
        DecisionTreeClassifier(max_depth=5),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
        AdaBoostClassifier()]



for index, clf in enumerate(clfs):
    clf.fit(X_train,y_train)
    preds = clf.predict(X_test)
    #print(names[index] + " Accuracy of the model is: %.2f%%" % (accuracy_score(preds,y_test) * 100))
    print("%.2f" % (accuracy_score(preds,y_test) * 100))



# print("NBC Accuracy of the model is: %.2f%%" % (accuracy_score(nbc_preds,y_test) * 100))
# print("MLP Accuracy of the model is: %.2f%%" % (accuracy_score(mlp_preds,y_test) * 100))
